## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Bethel,41.3712,-73.4140,77.86,74,20,6.91,clear sky,US,5/21/2022 14:50
1,1,Atuona,-9.8000,-139.0333,77.18,76,24,9.40,shower rain,PF,5/21/2022 14:50
2,2,The Valley,18.2170,-63.0578,85.10,74,20,13.80,clear sky,AI,5/21/2022 14:50
3,3,Liverpool,53.4106,-2.9779,66.38,63,75,10.36,clear sky,GB,5/21/2022 14:50
4,4,New Norfolk,-42.7826,147.0587,42.30,86,4,2.53,overcast clouds,AU,5/21/2022 14:50


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Bethel,41.3712,-73.4140,77.86,74,20,6.91,clear sky,US,5/21/2022 14:50
1,1,Atuona,-9.8000,-139.0333,77.18,76,24,9.40,shower rain,PF,5/21/2022 14:50
11,11,Opuwo,-18.0607,13.8400,77.74,8,0,4.72,shower rain,NaN,5/21/2022 14:51
19,19,Saint-Philippe,-21.3585,55.7679,76.59,84,43,12.80,overcast clouds,RE,5/21/2022 14:51
32,32,Bambous Virieux,-20.3428,57.7575,75.45,78,40,11.50,clear sky,MU,5/21/2022 14:51
33,33,Kapaa,22.0752,-159.3190,75.18,99,100,5.75,clear sky,US,5/21/2022 14:50
37,37,Komsomolskiy,40.4272,71.7189,79.52,29,100,16.11,broken clouds,UZ,5/21/2022 14:51
48,48,Mahebourg,-20.4081,57.7000,75.49,78,40,11.50,light intensity drizzle rain,MU,5/21/2022 14:48
64,64,Koulamoutou,-1.1303,12.4736,77.20,85,100,0.56,overcast clouds,GA,5/21/2022 14:51
75,75,Rio Muerto,27.4167,-110.2333,77.25,52,0,11.70,overcast clouds,MX,5/21/2022 14:51


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                106
City                   106
Lat                    106
Lng                    106
Max Temp               106
Humidity               106
Cloudiness             106
Wind Speed             106
Current Description    106
Country                105
Date                   106
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                105
City                   105
Lat                    105
Lng                    105
Max Temp               105
Humidity               105
Cloudiness             105
Wind Speed             105
Current Description    105
Country                105
Date                   105
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bethel,US,77.86,clear sky,41.3712,-73.4140,
1,Atuona,PF,77.18,shower rain,-9.8000,-139.0333,
19,Saint-Philippe,RE,76.59,overcast clouds,-21.3585,55.7679,
32,Bambous Virieux,MU,75.45,clear sky,-20.3428,57.7575,
33,Kapaa,US,75.18,clear sky,22.0752,-159.3190,
37,Komsomolskiy,UZ,79.52,broken clouds,40.4272,71.7189,
48,Mahebourg,MU,75.49,light intensity drizzle rain,-20.4081,57.7000,
64,Koulamoutou,GA,77.20,overcast clouds,-1.1303,12.4736,
75,Rio Muerto,MX,77.25,overcast clouds,27.4167,-110.2333,
94,Kisangani,CD,76.28,overcast clouds,0.5167,25.2000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping.")
        

Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ''] 
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bethel,US,77.86,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
1,Atuona,PF,77.18,shower rain,-9.8000,-139.0333,Villa Enata
19,Saint-Philippe,RE,76.59,overcast clouds,-21.3585,55.7679,Le four à pain
32,Bambous Virieux,MU,75.45,clear sky,-20.3428,57.7575,Casa Tia Villa
33,Kapaa,US,75.18,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))